In [1]:
import sys
sys.path.append("../code")
from dataset import build_dataset, build_dataloader
from config import DataArguments, TrainerArguments, ModelArguments

In [2]:
data_args = DataArguments()
training_args = TrainerArguments()
model_args = ModelArguments()

data_args.data_dir = "../data/"
data_args.asset_dir = "../assets/"
model_args.asset_dir = data_args.asset_dir
data_args.seed = training_args.seed
data_args.max_seq_len = model_args.max_seq_len

data_args.init_pct = 1
training_args.do_train = True
training_args.use_gpu = False

pool_dataset, train_dataset, model_args.vocab_size, model_args.num_labels = build_dataset(data_args, "train")
train_dataloader = build_dataloader(train_dataset, data_args)

[11/24/2021 20:59:19] INFO - dataset: Initialize Train Dataset.
[11/24/2021 20:59:19] INFO - dataset: Remove abstract.
[11/24/2021 20:59:19] INFO - dataset: Using cached dataset, wasn't able to remove columns.
[11/24/2021 20:59:19] INFO - dataset: Use the full dataset, for train dataset of total 47250 papers.
[11/24/2021 20:59:19] INFO - dataset: Train dataset was successfully initialized.
[11/24/2021 20:59:19] INFO - dataset: Successfully loaded mapper file ..\assets\area2idx.json
[11/24/2021 20:59:19] INFO - dataset: Use tokenized_paperswithcode dataset.
[11/24/2021 20:59:19] INFO - dataset: Preprocessed dataset. Use default vocab_size=83931.
[11/24/2021 20:59:19] INFO - dataset: Successfully converted dataset to dataloader.


In [ ]:
from trainer import NaiveTrainer
trainer= NaiveTrainer(
    data_args,
    training_args,
    model_args,
    training_dataset=train_dataloader if training_args.do_train else None,
    validation_dataset=valid_dataloader if training_args.do_valid else None,
    test_dataset=test_dataloader if training_args.do_test else None,
)

[11/24/2021 16:55:46] INFO - trainer: Bert was selected. Start Transformer setup.
[11/24/2021 16:55:46] INFO - trainer: Successfully setup transformer settings.


In [14]:
(train_dataset[0, 4]["input_ids"] == 2).nonzero()[:, 1]

tensor([17, 14])

In [19]:
batch = next(iter(train_dataloader))

In [20]:
import torch
trainer.model.load_state_dict(torch.load("../output/BERT (1)/checkpoints/099.pt"))
# trainer.model.load_state_dict(torch.load("../output/BERT_BCE (4)/checkpoints/014.pt"))

<All keys matched successfully>

In [41]:
pool_dataset

In [47]:
import math
from dataclasses import dataclass
from typing import List

import torch
from toma import toma
from tqdm.auto import tqdm

from batchbald_redux import joint_entropy

K = 20

In [48]:
import numpy as np


def get_mixture_prob_dist(p1, p2, m):
    return (1.0 - m) * np.asarray(p1) + m * np.asarray(p2)


p1 = [0.7, 0.1, 0.1, 0.1]
p2 = [0.3, 0.3, 0.2, 0.2]
y1_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.7, 0.1, 0.1]
p2 = [0.2, 0.3, 0.3, 0.2]
y2_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.1, 0.7, 0.1]
p2 = [0.2, 0.2, 0.3, 0.3]
y3_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.1, 0.1, 0.7]
p2 = [0.3, 0.2, 0.2, 0.3]
y4_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]


def nested_to_tensor(l):
    return torch.stack(list(map(torch.as_tensor, l)))


ys_ws = nested_to_tensor([y1_ws, y2_ws, y3_ws, y4_ws])

In [49]:
ys_ws

tensor([[[0.7000, 0.1000, 0.1000, 0.1000],
         [0.6789, 0.1105, 0.1053, 0.1053],
         [0.6579, 0.1211, 0.1105, 0.1105],
         [0.6368, 0.1316, 0.1158, 0.1158],
         [0.6158, 0.1421, 0.1211, 0.1211],
         [0.5947, 0.1526, 0.1263, 0.1263],
         [0.5737, 0.1632, 0.1316, 0.1316],
         [0.5526, 0.1737, 0.1368, 0.1368],
         [0.5316, 0.1842, 0.1421, 0.1421],
         [0.5105, 0.1947, 0.1474, 0.1474],
         [0.4895, 0.2053, 0.1526, 0.1526],
         [0.4684, 0.2158, 0.1579, 0.1579],
         [0.4474, 0.2263, 0.1632, 0.1632],
         [0.4263, 0.2368, 0.1684, 0.1684],
         [0.4053, 0.2474, 0.1737, 0.1737],
         [0.3842, 0.2579, 0.1789, 0.1789],
         [0.3632, 0.2684, 0.1842, 0.1842],
         [0.3421, 0.2789, 0.1895, 0.1895],
         [0.3211, 0.2895, 0.1947, 0.1947],
         [0.3000, 0.3000, 0.2000, 0.2000]],

        [[0.1000, 0.7000, 0.1000, 0.1000],
         [0.1053, 0.6789, 0.1105, 0.1053],
         [0.1105, 0.6579, 0.1211, 0.1105],
         

In [90]:
def to_2d(logits):

    if logits.ndim == 3 and logits.shape[1] == 1:
        logits = logits.squeeze()
        return logits
    elif logits.ndim == 3 and logits.shape[1] > 1:
        raise


In [16]:
import torch
import numpy as np

p1 = [0.7, 0.1, 0.1, 0.1, 0.0]
p2 = [0.3, 0.3, 0.2, 0.2, 0.0]

logits = torch.stack(tuple(map(torch.tensor, (p1, p2)))).unsqueeze(1)
logits_np = np.array((p1, p2))

In [25]:
from acquisition_function import *

In [31]:
# lst = []
lst.append(margin_of_confidence(logits))

In [36]:
torch.cat(lst).argsort().tolist()

[0, 2, 1, 3]

In [117]:
part = torch.split(-_logits, 1, dim=1)[0].squeeze()

In [118]:
part

tensor([-0.7000, -0.3000])

In [73]:
part[:, 1]

tensor([-0.1000, -0.3000])

In [122]:
sorted_logit = logits.squeeze().sort(dim=1).values

In [19]:
from acquisition_function import margin_of_confidence

In [20]:
margin_of_confidence(logits)

tensor([0.6000, 0.0000])

In [24]:
len(train_dataset[:50000]["input_ids"])

47250